In [27]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_season(year):
    # cloudscraper automatically handles cloudflare and bot detection
    scraper = cloudscraper.create_scraper(
        browser={
            'browser': 'chrome',
            'platform': 'windows',
            'desktop': True
        }
    )
    
    url = f"https://www.basketball-reference.com/leagues/NBA_2013_totals.html#all_totals_stats"
    
    # Add retry logic
    max_retries = 3
    for attempt in range(max_retries):
        try:
            r = scraper.get(url, timeout=15)
            r.raise_for_status()
            break
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  Attempt {attempt + 1} failed, retrying in 10 seconds...")
                time.sleep(10)
            else:
                raise

    soup = BeautifulSoup(r.text, "html.parser")
    table = soup.find("table", {"id": "totals"})



    return soup, table

In [ ]:
    rows = []
    for tr in table.tbody.find_all("tr"):
        # skip separator rows
        if "class" in tr.attrs and "thead" in tr["class"]:
            continue

        tds = tr.find_all("td")
        if not tds:
            continue

        player = tr.find("td", {"data-stat": "player"}).text.strip()
        team = tr.find("td", {"data-stat": "team_id"}).text.strip()
        games = tr.find("td", {"data-stat": "g"}).text.strip()

        rows.append({
            "player": player,
            "team": team,
            "games_played": int(games),
            "year": year
        })

In [28]:
soup, table = scrape_season(1991)

In [33]:
soup.find("table")

<table class="stats_table sortable soc" data-cols-to-freeze="2,4" data-non-qual="1" data-qual-label=" When table is sorted, hide non-qualifiers for rate stats" data-qual-text="" data-soc-sum-params="null" data-soc-sum-phase-type="reg" data-soc-sum-scope-type="player_season" id="totals_stats"> <caption>Totals Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col class="sort_col"/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rk" class="ranker poptip center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th> <th aria-label="Player" class="poptip sort_default_asc center" data-stat="name_display" scope="col">Player</th> <th aria-label="Age" class="poptip center" data-stat="age" data-tip="Player's age on January 31 of the season" scope="col">Age</th> <th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_name_

In [11]:
# years you want to scrape
years = list(range(1991, 2023))  # adjust to match your salary file

all_seasons = []

for y in years:
    print(f"Scraping {y}…")
    df = scrape_season(y)
    all_seasons.append(df)
    time.sleep(3)  # increased delay to be more polite

master = pd.concat(all_seasons, ignore_index=True)

# keep the team with the most games for each (player, year)
master_main_team = (
    master.sort_values(["player", "year", "games_played"], ascending=[True, True, False])
          .drop_duplicates(subset=["player", "year"], keep="first")
)

Scraping 1991…


AttributeError: 'NoneType' object has no attribute 'tbody'

In [ ]:
master_main_team.to_csv("nba_player_main_team_by_year.csv", index=False)
print("Saved nba_player_main_team_by_year.csv")